In [15]:
import yfinance as yf
import boto
from os import environ as env

In [16]:

def calc_stock(high, current):
    return (current - high)/high


def convert_tuple(tup): 
    string_tup =  "{} : {}".format(tup[0], str(tup[1]))
    return string_tup


def create_message(pairs):
    message = "Ordered Portfolio:\n"
    for pair in d:
        message += convert_tuple(pair) + "\n"
    
    
def publish_message_sns(message):
    sns_arn = env.get('SNS_ARN').strip()
    sns_client = boto3.client('sns')
    try:
        print(message)

        response = sns_client.publish(
            TopicArn=sns_arn,
            Message=message
        )

        print(response)

    except Exception as e:
        print("ERROR PUBLISHING MESSAGE TO SNS: {}".format(e))


def read_tickers(file='stock_tickers.txt'):
      
    pairs = dict()

    with open(file, 'r') as f:
        while True:
            # Get next line from file 
            ticker = (f.readline()).strip()
            
            if ticker == "":
                break

            stock = yf.Ticker(ticker)
            data = stock.history('5y')
            
            close = data.Close[-1]
            high = max(data.Close)
            
            delta = calc_stock(high, close)
 
            pairs[ticker] = delta

            if not ticker:
                break
        
    return(sorted(pairs.items(), key=lambda x: x[1]))


def handler():
    pairs = read_tickers()
    messsage = create_message(pairs)
    print(message)
    #AWS LIVE VERSION ONLY:
    #publish_message_sns(message)
    

In [17]:
handler()

Ordered Portfolio:

UCO : -0.948176583493282
COF : -0.49346497414198404
WFC : -0.4863611059565782
C : -0.45913727417967315
AXP : -0.3830505985165601
BAC : -0.3645862750635413
JPM : -0.3390270270270271
